# Train a neural network model to match the X to the Y

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
X = np.array([-4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0], dtype=float)
Y = np.array([5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0], dtype=float)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=20, input_shape=[1]),
    tf.keras.layers.Dense(units=15),
    tf.keras.layers.Dense(units=10),
    tf.keras.layers.Dense(units=1)
    ])

In [ ]:
model.compile(optimizer='sgd', loss='mean_squared_error')

In [ ]:
model.fit(X, Y, epochs=500)

Epoch 1/500
1/1 [==============================] - 0s 435ms/step - loss: 110.6602
Epoch 2/500
1/1 [==============================] - 0s 15ms/step - loss: 97.0252
Epoch 3/500
1/1 [==============================] - 0s 10ms/step - loss: 86.8461
Epoch 4/500
1/1 [==============================] - 0s 9ms/step - loss: 79.0747
Epoch 5/500
1/1 [==============================] - 0s 9ms/step - loss: 72.9887
Epoch 6/500
1/1 [==============================] - 0s 9ms/step - loss: 68.0904
Epoch 7/500
1/1 [==============================] - 0s 10ms/step - loss: 64.0362
Epoch 8/500
1/1 [==============================] - 0s 10ms/step - loss: 60.5889
Epoch 9/500
1/1 [==============================] - 0s 14ms/step - loss: 57.5842
Epoch 10/500
1/1 [==============================] - 0s 12ms/step - loss: 54.9082
Epoch 11/500
1/1 [==============================] - 0s 11ms/step - loss: 52.4816
Epoch 12/500
1/1 [==============================] - 0s 8ms/step - loss: 50.2489
Epoch 13/500
1/1 [=====================

In [ ]:
model.predict([4, 5]).round()

1/1 [==============================] - 0s 218ms/step


array([[13.],
       [14.]], dtype=float32)